In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
#m6 = pd.read_csv(r'..\Data\Clean\carchive_M6.csv')

In [5]:
#nrtmodis = pd.read_csv(r'..\Data\Clean\cnrt_M6.csv')

In [6]:
rain = pd.read_csv(r'..\Data\Clean\rain.csv')

In [7]:
wind = pd.read_csv(r'..\Data\Clean\wind.csv')

In [8]:
weather = pd.read_csv(r'..\Data\Raw\weatherAUS.csv')

In [9]:
import os
os.getcwd()

'D:\\Dokumente\\GitHub\\AustralienFiresFromSpace_Schoeggl_Vorauer\\Notebooks'